In [5]:
# combine_dustmaps.py: make a composite extinction map (Marshall,Green,Drimmel)
# Here we combine several dustmaps for a fast access in mock catalogue generation
# Marshall06 in their respective footprint
# Bayestar 19 for |b|<=20
# Bayestar 17 for |b|>20
# Drimmel 03 elsewhere
# The map is a 2darray sorted by nested healpix number of nside = 512
# and the second axis gives the A0 value for different distances
# We use the same distance spacing as Bayestar 19, i.e. np.linspace(4,18.875,120) in distance modulus
###############################################################################
import numpy as np
import healpy as hp
import mwdust
import dustmaps
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
def distmod2dist(distmod):
    """distance modulus to distance in kpc"""
    return 10.**(distmod/5.-2.)

def get_nside(healpix_number = 1):
    """
    returns the nside for specific healpix level
    """
    return(np.power(2,healpix_number))

def get_hpx_number(nside):
    """
    returns hpx level for given nside
    """
    return(int(np.log2(nside)))

def number_of_healpixels(healpix_number = 1):
    """
    returns the number of pixels for a specific level
    """
    return(np.power(4,healpix_number)*12)

_GREEN15DISTMODS= np.linspace(4.,18.875,120)
#_GREEN15DISTMODS= np.linspace(4.,19,31)
_GREEN15DISTS= distmod2dist(_GREEN15DISTMODS)
dists = _GREEN15DISTS
print(dists)

[ 0.06309573  0.06683439  0.07079458  0.07498942  0.07943282  0.08413951
  0.08912509  0.09440609  0.1         0.10592537  0.11220185  0.11885022
  0.12589254  0.13335214  0.14125375  0.14962357  0.15848932  0.1678804
  0.17782794  0.18836491  0.19952623  0.2113489   0.22387211  0.23713737
  0.25118864  0.26607251  0.28183829  0.29853826  0.31622777  0.33496544
  0.35481339  0.3758374   0.39810717  0.4216965   0.44668359  0.47315126
  0.50118723  0.53088444  0.56234133  0.59566214  0.63095734  0.66834392
  0.70794578  0.74989421  0.79432823  0.84139514  0.89125094  0.94406088
  1.          1.05925373  1.12201845  1.18850223  1.25892541  1.33352143
  1.41253754  1.49623566  1.58489319  1.67880402  1.77827941  1.88364909
  1.99526231  2.11348904  2.23872114  2.37137371  2.51188643  2.66072506
  2.81838293  2.98538262  3.16227766  3.34965439  3.54813389  3.75837404
  3.98107171  4.21696503  4.46683592  4.73151259  5.01187234  5.30884444
  5.62341325  5.95662144  6.30957344  6.68343918  7.

In [6]:
get_hpx_number(16)

4

In [2]:
#from mwdust import Combined15 ,Combined19
#from mwdust import Green15, Green17, Green19
from dustmaps import bayestar
from mwdust import Combined19
#from mwdust import Green17
com19 = Combined19()
#com15 = Combined15()
#gr15 = Green15()
#gr17 = Green17()
#gr19 = Green19()
bs = bayestar.BayestarQuery(version = "bayestar2017", max_samples=1)

Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 39.282 s
  pix_info:   0.489 s
   samples:  32.045 s
      best:   2.013 s
       nan:   0.038 s
      sort:   4.565 s
       idx:   0.132 s


In [3]:
for hpx_level in [1,2,3,4,5,6,7,8,9]:
    nside = get_nside(hpx_level)
    print(nside)
    nHpx = number_of_healpixels(hpx_level)
    print(nHpx)
    ext = np.zeros(shape = (nHpx,len(dists)), dtype = np.float16)
    for i in range(nHpx):
        if i%1000 == 0:
            print(i, nHpx)
        lb = hp.pix2ang(nside,i,nest = True, lonlat = True)
        if np.abs(lb[1]) <= 20:
            ext[i] = com19(lb[0],lb[1],dists)
        else:
            coords = SkyCoord(lb[0]*np.ones_like(dists)*u.deg, lb[1]*np.ones_like(dists)*u.deg, dists*u.kpc, frame='galactic')
            ext[i] = bs(coords)
            if np.all(np.isnan(ext[i])):
                ext[i] = com19(lb[0],lb[1],dists)
    np.save("combined1719_hpxlevel_%d.npy" %(hpx_level), ext)

2
48
0 48
4
192
0 192
8
768
0 768
16
3072
0 3072
1000 3072
2000 3072
3000 3072
32
12288
0 12288
1000 12288
2000 12288
3000 12288
4000 12288
5000 12288
6000 12288
7000 12288
8000 12288
9000 12288
10000 12288
11000 12288
12000 12288
64
49152
0 49152
1000 49152
2000 49152
3000 49152
4000 49152
5000 49152
6000 49152
7000 49152
8000 49152
9000 49152
10000 49152
11000 49152
12000 49152
13000 49152
14000 49152
15000 49152
16000 49152
17000 49152
18000 49152
19000 49152
20000 49152
21000 49152
22000 49152
23000 49152
24000 49152
25000 49152
26000 49152
27000 49152
28000 49152
29000 49152
30000 49152
31000 49152
32000 49152
33000 49152
34000 49152
35000 49152
36000 49152
37000 49152
38000 49152
39000 49152
40000 49152
41000 49152
42000 49152
43000 49152
44000 49152
45000 49152
46000 49152
47000 49152
48000 49152
49000 49152
128
196608
0 196608
1000 196608
2000 196608
3000 196608
4000 196608
5000 196608
6000 196608
7000 196608
8000 196608
9000 196608
10000 196608
11000 196608
12000 196608
13000 

344000 786432
345000 786432
346000 786432
347000 786432
348000 786432
349000 786432
350000 786432
351000 786432
352000 786432
353000 786432
354000 786432
355000 786432
356000 786432
357000 786432
358000 786432
359000 786432
360000 786432
361000 786432
362000 786432
363000 786432
364000 786432
365000 786432
366000 786432
367000 786432
368000 786432
369000 786432
370000 786432
371000 786432
372000 786432
373000 786432
374000 786432
375000 786432
376000 786432
377000 786432
378000 786432
379000 786432
380000 786432
381000 786432
382000 786432
383000 786432
384000 786432
385000 786432
386000 786432
387000 786432
388000 786432
389000 786432
390000 786432
391000 786432
392000 786432
393000 786432
394000 786432
395000 786432
396000 786432
397000 786432
398000 786432
399000 786432
400000 786432
401000 786432
402000 786432
403000 786432
404000 786432
405000 786432
406000 786432
407000 786432
408000 786432
409000 786432
410000 786432
411000 786432
412000 786432
413000 786432
414000 786432
415000

140000 3145728
141000 3145728
142000 3145728
143000 3145728
144000 3145728
145000 3145728
146000 3145728
147000 3145728
148000 3145728
149000 3145728
150000 3145728
151000 3145728
152000 3145728
153000 3145728
154000 3145728
155000 3145728
156000 3145728
157000 3145728
158000 3145728
159000 3145728
160000 3145728
161000 3145728
162000 3145728
163000 3145728
164000 3145728
165000 3145728
166000 3145728
167000 3145728
168000 3145728
169000 3145728
170000 3145728
171000 3145728
172000 3145728
173000 3145728
174000 3145728
175000 3145728
176000 3145728
177000 3145728
178000 3145728
179000 3145728
180000 3145728
181000 3145728
182000 3145728
183000 3145728
184000 3145728
185000 3145728
186000 3145728
187000 3145728
188000 3145728
189000 3145728
190000 3145728
191000 3145728
192000 3145728
193000 3145728
194000 3145728
195000 3145728
196000 3145728
197000 3145728
198000 3145728
199000 3145728
200000 3145728
201000 3145728
202000 3145728
203000 3145728
204000 3145728
205000 3145728
206000 314

687000 3145728
688000 3145728
689000 3145728
690000 3145728
691000 3145728
692000 3145728
693000 3145728
694000 3145728
695000 3145728
696000 3145728
697000 3145728
698000 3145728
699000 3145728
700000 3145728
701000 3145728
702000 3145728
703000 3145728
704000 3145728
705000 3145728
706000 3145728
707000 3145728
708000 3145728
709000 3145728
710000 3145728
711000 3145728
712000 3145728
713000 3145728
714000 3145728
715000 3145728
716000 3145728
717000 3145728
718000 3145728
719000 3145728
720000 3145728
721000 3145728
722000 3145728
723000 3145728
724000 3145728
725000 3145728
726000 3145728
727000 3145728
728000 3145728
729000 3145728
730000 3145728
731000 3145728
732000 3145728
733000 3145728
734000 3145728
735000 3145728
736000 3145728
737000 3145728
738000 3145728
739000 3145728
740000 3145728
741000 3145728
742000 3145728
743000 3145728
744000 3145728
745000 3145728
746000 3145728
747000 3145728
748000 3145728
749000 3145728
750000 3145728
751000 3145728
752000 3145728
753000 314

1219000 3145728
1220000 3145728
1221000 3145728
1222000 3145728
1223000 3145728
1224000 3145728
1225000 3145728
1226000 3145728
1227000 3145728
1228000 3145728
1229000 3145728
1230000 3145728
1231000 3145728
1232000 3145728
1233000 3145728
1234000 3145728
1235000 3145728
1236000 3145728
1237000 3145728
1238000 3145728
1239000 3145728
1240000 3145728
1241000 3145728
1242000 3145728
1243000 3145728
1244000 3145728
1245000 3145728
1246000 3145728
1247000 3145728
1248000 3145728
1249000 3145728
1250000 3145728
1251000 3145728
1252000 3145728
1253000 3145728
1254000 3145728
1255000 3145728
1256000 3145728
1257000 3145728
1258000 3145728
1259000 3145728
1260000 3145728
1261000 3145728
1262000 3145728
1263000 3145728
1264000 3145728
1265000 3145728
1266000 3145728
1267000 3145728
1268000 3145728
1269000 3145728
1270000 3145728
1271000 3145728
1272000 3145728
1273000 3145728
1274000 3145728
1275000 3145728
1276000 3145728
1277000 3145728
1278000 3145728
1279000 3145728
1280000 3145728
1281000 

1732000 3145728
1733000 3145728
1734000 3145728
1735000 3145728
1736000 3145728
1737000 3145728
1738000 3145728
1739000 3145728
1740000 3145728
1741000 3145728
1742000 3145728
1743000 3145728
1744000 3145728
1745000 3145728
1746000 3145728
1747000 3145728
1748000 3145728
1749000 3145728
1750000 3145728
1751000 3145728
1752000 3145728
1753000 3145728
1754000 3145728
1755000 3145728
1756000 3145728
1757000 3145728
1758000 3145728
1759000 3145728
1760000 3145728
1761000 3145728
1762000 3145728
1763000 3145728
1764000 3145728
1765000 3145728
1766000 3145728
1767000 3145728
1768000 3145728
1769000 3145728
1770000 3145728
1771000 3145728
1772000 3145728
1773000 3145728
1774000 3145728
1775000 3145728
1776000 3145728
1777000 3145728
1778000 3145728
1779000 3145728
1780000 3145728
1781000 3145728
1782000 3145728
1783000 3145728
1784000 3145728
1785000 3145728
1786000 3145728
1787000 3145728
1788000 3145728
1789000 3145728
1790000 3145728
1791000 3145728
1792000 3145728
1793000 3145728
1794000 

2245000 3145728
2246000 3145728
2247000 3145728
2248000 3145728
2249000 3145728
2250000 3145728
2251000 3145728
2252000 3145728
2253000 3145728
2254000 3145728
2255000 3145728
2256000 3145728
2257000 3145728
2258000 3145728
2259000 3145728
2260000 3145728
2261000 3145728
2262000 3145728
2263000 3145728
2264000 3145728
2265000 3145728
2266000 3145728
2267000 3145728
2268000 3145728
2269000 3145728
2270000 3145728
2271000 3145728
2272000 3145728
2273000 3145728
2274000 3145728
2275000 3145728
2276000 3145728
2277000 3145728
2278000 3145728
2279000 3145728
2280000 3145728
2281000 3145728
2282000 3145728
2283000 3145728
2284000 3145728
2285000 3145728
2286000 3145728
2287000 3145728
2288000 3145728
2289000 3145728
2290000 3145728
2291000 3145728
2292000 3145728
2293000 3145728
2294000 3145728
2295000 3145728
2296000 3145728
2297000 3145728
2298000 3145728
2299000 3145728
2300000 3145728
2301000 3145728
2302000 3145728
2303000 3145728
2304000 3145728
2305000 3145728
2306000 3145728
2307000 

2758000 3145728
2759000 3145728
2760000 3145728
2761000 3145728
2762000 3145728
2763000 3145728
2764000 3145728
2765000 3145728
2766000 3145728
2767000 3145728
2768000 3145728
2769000 3145728
2770000 3145728
2771000 3145728
2772000 3145728
2773000 3145728
2774000 3145728
2775000 3145728
2776000 3145728
2777000 3145728
2778000 3145728
2779000 3145728
2780000 3145728
2781000 3145728
2782000 3145728
2783000 3145728
2784000 3145728
2785000 3145728
2786000 3145728
2787000 3145728
2788000 3145728
2789000 3145728
2790000 3145728
2791000 3145728
2792000 3145728
2793000 3145728
2794000 3145728
2795000 3145728
2796000 3145728
2797000 3145728
2798000 3145728
2799000 3145728
2800000 3145728
2801000 3145728
2802000 3145728
2803000 3145728
2804000 3145728
2805000 3145728
2806000 3145728
2807000 3145728
2808000 3145728
2809000 3145728
2810000 3145728
2811000 3145728
2812000 3145728
2813000 3145728
2814000 3145728
2815000 3145728
2816000 3145728
2817000 3145728
2818000 3145728
2819000 3145728
2820000 

In [4]:
"""
from mwdust import Green17
from dustmaps import bayestar
gr17 = Green17()
bs = bayestar.BayestarQuery(version = "bayestar2017", max_samples=1)
i = 1000
lb = hp.pix2ang(256,i,nest = True, lonlat = True)
l = 270.
b = 1.
coords = SkyCoord(l*u.deg, b*u.deg, dists*u.kpc, frame='galactic')
#print(com15(l,b,dists))
#print(com19(l,b,dists))
#print(gr15(l,b,dists))
#print(gr17(l,b,dists))
#print(gr19(l,b,dists))
print(bs(coords))
"""

'\nfrom mwdust import Green17\nfrom dustmaps import bayestar\ngr17 = Green17()\nbs = bayestar.BayestarQuery(version = "bayestar2017", max_samples=1)\ni = 1000\nlb = hp.pix2ang(256,i,nest = True, lonlat = True)\nl = 270.\nb = 1.\ncoords = SkyCoord(l*u.deg, b*u.deg, dists*u.kpc, frame=\'galactic\')\n#print(com15(l,b,dists))\n#print(com19(l,b,dists))\n#print(gr15(l,b,dists))\n#print(gr17(l,b,dists))\n#print(gr19(l,b,dists))\nprint(bs(coords))\n'